In [8]:
from googleapiclient.discovery import build
import pandas as pd

In [ ]:
# IMPORTANTE: Use variáveis de ambiente para proteger sua chave de API
# Configure a variável de ambiente antes de executar:
# export YOUTUBE_API_KEY="sua_chave_aqui"
# Ou crie um arquivo .env com: YOUTUBE_API_KEY=sua_chave_aqui

import os
from dotenv import load_dotenv

# Carregar variáveis de ambiente do arquivo .env (se existir)
load_dotenv()

# Obter a chave da API do ambiente
API_KEY = os.getenv("YOUTUBE_API_KEY")

if not API_KEY:
    raise ValueError(
        "Chave de API não encontrada! "
        "Configure a variável de ambiente YOUTUBE_API_KEY ou crie um arquivo .env. "
        "Veja o README.md para instruções."
    )

# Criar o cliente da API
youtube = build("youtube", "v3", developerKey=API_KEY)

In [10]:
# Buscar vídeos
request = youtube.search().list(
    part="snippet",
    q="eleição",
    type="video",
    regionCode="BR",          # restringe ao Brasil
    location=" -23.5505, -46.6333",  # coordenadas de São Paulo
    locationRadius="50km",    # raio da busca
    maxResults=50,
    publishedAfter="2022-10-29T00:00:00Z",  # véspera do 2º turno
    publishedBefore="2022-10-31T00:00:00Z"  # logo após o 2º turno
)

response = request.execute()

In [11]:
# Extrair IDs dos vídeos
video_ids = [item["id"]["videoId"] for item in response["items"]]

In [12]:
# --- Buscar estatísticas desses vídeos ---
stats_request = youtube.videos().list(
    part="snippet,statistics",
    id=",".join(video_ids)
)

stats_response = stats_request.execute()

In [13]:
# Extrair dados básicos
videos = []
for item in response["items"]:
    videos.append({
        "videoId": item["id"]["videoId"],
        "titulo": item["snippet"]["title"],
        "canal": item["snippet"]["channelTitle"],
        "publicado_em": item["snippet"]["publishedAt"]
    })



In [14]:
df = pd.DataFrame(videos)
print(df)

        videoId                                             titulo  \
0   3YG062ig0Fo  Sobre Violência, Abuso, Eleição e Democracia, ...   
1   w8ZsSWITza4         30/10/2022 Eleição que ficará na história!   
2   7K-TXb8vHQo                   Eleições #eleições #eleições2022   
3   rguqzieAjGU                     Borra Eleição e Futuro Mandato   
4   2SvFhfGhwr0           Tribunal Regional Eleitoral de São Paulo   
5   bpA_B5UFcGs                 VALORANT - DOMINGO DE ELEIÇÃO 2022   
6   PyCHcctfXnM  [FullHD] Encerramento do &quot;Agora CNN - Esp...   
7   PYrpygTR3es  Alegria dia de eleição Palmeiras campeão.  vot...   
8   8YlQbJKQk9k  A PEDIDOS SOBRE O QUE VEM APOS RESULTADO ELEIÇ...   
9   e7Xah-QhjUc                      eu indo voltar dia eleições 💋   
10  GEUd9Bq67xA  ALEXANDRE DE MORAES SE PRONUNCIA SOBRE OPERAÇÃ...   
11  pj_Nr2b1p0E  [HD] Início do “CNN Eleições 2ª Edição” - CNN ...   
12  AzD2d9wDYQM  [HD] Trechos do &quot;Jornal da CNN - Especial...   
13  fmXejApEeUo  [Fu

In [15]:
# Organizar em DataFrame
videos_info = []
for item in stats_response["items"]:
    stats = item["statistics"]
    videos_info.append({
        "videoId": item["id"],
        "titulo": item["snippet"]["title"],
        "canal": item["snippet"]["channelTitle"],
        "publicado_em": item["snippet"]["publishedAt"],
        "views": int(stats.get("viewCount", 0)),
        "likes": int(stats.get("likeCount", 0)),
        "comentarios": int(stats.get("commentCount", 0))
    })

df = pd.DataFrame(videos_info)

# --- Descobrir o vídeo com mais comentários ---
top_video = df.sort_values(by="comentarios", ascending=False).iloc[0]

print("🎥 Vídeo com mais comentários:")
print(top_video)

🎥 Vídeo com mais comentários:
videoId                                               buWjQ87OQM0
titulo          DEBATE NA GLOBO: LULA CHAMA BOLSONARO DE DESCO...
canal                                                   TV Afiada
publicado_em                                 2022-10-29T01:19:15Z
views                                                       33146
likes                                                        4686
comentarios                                                   393
Name: 17, dtype: object
